In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import timm
import torch
import glob
import os
import torchvision.transforms as transforms
from typing import Literal
from PIL import Image

from lib.driving_dataset.DrivingDataset import DrivingDataset, Preprocessor

/home/ashmitchamoli/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
def loadImages(videoName : str, split : Literal['train', 'dev', 'test']) -> DrivingDataset:
	pass

In [31]:
trainDataset = DrivingDataset('test_video', 'train')
model = timm.create_model('mobilenetv3_large_100', pretrained=True, num_classes=trainDataset.numClasses)

In [34]:
videoName = 'test_video'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

imagePaths = glob.glob(f'./Datasets/SynDDAnnotated/{videoName}/frames/*.jpg')
print(f"Loaded {len(imagePaths)} images")

def getLabel(imagePath): 
    class_label = imagePath.split('/')[-1].split('_')[1]
    class_label = int(float(class_label)) if class_label != "nan" else 0  
    return class_label

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        label = getLabel(image_path)
        return image, label
    
# Define the transformation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Creating the dataset and dataloader")
# Create the dataset
dataset = ImageDataset(imagePaths, transform=transform)

print("Creating the dataloader")
# Make train, test, validation splits
train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
val_size = len(dataset) - train_size - test_size
train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size, val_size])

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=True)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()

# Only train the classifier parameters
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

print("Training the model")
# Train the model
model.train()
model.to(device)

for epoch in range(5):
    print(f"Epoch {epoch + 1}")
    running_loss = 0.0
    total = 0
    correct = 0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # if i % 20 == 0:
        #     print(f"Train Loss: {running_loss / 20}")
        #     print(f"Train Accuracy: {100 * correct / total}%")
        #     running_loss = 0.0
        #     total = 0
        #     correct = 0
    running_loss = running_loss / len(train_dataloader)
    print(f"Train Loss: {running_loss}")
    print(f"Train Accuracy: {100 * correct / total}%")

    for i, data in enumerate(val_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total}%")

print("Finished Training")

Loaded 10770 images
Creating the dataset and dataloader
Creating the dataloader
Training the model
Epoch 1
Train Loss: 0.14908532349912912
Train Accuracy: 96.46007428040855%
Validation Accuracy: 96.64706489219024%
Epoch 2
Train Loss: 0.06959705077451239
Train Accuracy: 98.46796657381616%
Validation Accuracy: 98.545341999381%
Epoch 3
Train Loss: 0.04158960040345988
Train Accuracy: 99.21077065923862%
Validation Accuracy: 99.2056122975343%
Epoch 4
Train Loss: 0.03294906490897498
Train Accuracy: 99.21077065923862%
Validation Accuracy: 99.21592902094295%
Epoch 5
Train Loss: 0.024641239710160717
Train Accuracy: 99.45450324976787%
Validation Accuracy: 99.48416382956773%
Finished Training


AttributeError: 'MobileNetV3' object has no attribute 'module'

In [35]:
# Save the model
os.makedirs("./models/", exist_ok=True)
torch.save(model.state_dict(), "./models/classifier.pth")

# Test the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in test_dataloader:
        images, labels = data[0].to(device), data[1].to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the {total} test images: {100 * correct / total}%")

Accuracy of the network on the 1077 test images: 99.44289693593315%
